# **Homework 5: Convolutional Neural Networks**

---

## **Setup**

### **Change Colab's Runtime**

Colab provides machine instances in the cloud. We will use Colab to speed up training times via its GPUs.
By default, Colab instances do not use GPUs. So, we must first enable GPU support.

1. Click the "Runtime" menu above
2. Click "Change runtime type"
3. Select "GPU" under "Hardware accelerator"
4. Click Save

## **Linking Github and Google Colab**

We're going to create a public/private key pair to allow Colab to access your Github.

1. Make sure your latest code is pushed to github.com
2. Run the cell below by clicking the 'Play' button to the top left.
  *   Google Colab will start a machine instance in the cloud (top right of Colab under 'Share')
  *   When the script asks for the location, click the flashing cursor and press enter
  *   Leave the password field blank

In [ ]:
!ssh-keygen -t rsa
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
!clear
!echo "PUBLIC KEY: (for github.com)"
!cat /root/.ssh/id_rsa.pub

Follow these steps before proceeding:

1. Go to https://github.com/settings/keys and login if necessary
2. Click the green "New SSH Key" button
3. Choose any title
4. From the cell above in this Colab notebook, copy the public key. It looks like "ssh-rsa xxxxx", is very long, and should be the last line.
5. Paste the key into the "key" text box and click "Add SSH key"
6. Below, replace **[GITHUBUSERNAME]** with your GitHub username.
7. Run the cell.

In [ ]:
!git clone git@github.com:BrownCSCI1430/homework5_cnns-[GITHUBUSERNAME].git
%cd homework5_cnns-[GITHUBUSERNAME]
!git config --global user.email "colab_bot@brown.edu"
!git config --global user.name "Colab Bot"

Next, note the left-hand side bar.
Find and click on the "Files" button on the left; it looks like a file divider. Now, you should be able to see your homework5_cnn folder.

## **Editing Your Code**

Your code on Colab must be pushed back to Github to reflect changes. Thus, we recommend changing your code and tuning parameters **either locally or on Colab, but not both simultaneously**.
This will help avoid merge conflicts, which will happen when you make changes without ensuring that you have an up-to-date copy of the code.

#### Workflow 1

1. Make change locally in `vscode`
2. git commit+push to Github in your terminal
3. git pull from Github into the Colab copy of your project
4. Execute code in Colab

#### Workflow 2

1. Make changes to your project files on Colab
2. git commit+push to Github using Colab cells
3. git pull from Github into your local directory on your laptop filesystem
4. Edit in `vscode`

### **Workflow 1: Making changes locally**

If you choose to change your files locally, **DO NOT repeatedly clone the repo**. Once you pushed from local terminal, you can update your code on colab using the following command.

In [ ]:
!git pull

### **Workflow 2: Making changes on Colab**

If you choose to change your files on Colab, you can access them from the files section of the left sidebar. You may double click on the file you wish to edit and open it on colab.

<img src="https://drive.google.com/uc?id=15fusCIFNpGiibzg5FWod4o8DlpjNHJMZ" alt="colab folder" width="80%">

You can use the following commands to push your code changes to github from colab:

In [ ]:
!git add code/*
!git commit -m "updating code from colab!"
!git push

### **Installation**

We can now install and import the necessary components.

In [18]:
%pip install lime

Note: you may need to restart the kernel to use updated packages.


In [19]:
import tensorflow as tf
from tensorflow.keras.layers import \
    Conv2D, MaxPool2D, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
from skimage.io import imread

Next, we can download the weights for the VGG model.

In [1]:
!wget "https://browncsci1430.github.io/hw5_cnns/vgg16_imagenet.h5"

--2024-05-10 02:34:23--  https://browncsci1430.github.io/hw5_cnns/vgg16_imagenet.h5
Resolving browncsci1430.github.io (browncsci1430.github.io)... 2606:50c0:8003::153, 2606:50c0:8001::153, 2606:50c0:8000::153, ...
Connecting to browncsci1430.github.io (browncsci1430.github.io)|2606:50c0:8003::153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58909648 (56M) [application/octet-stream]
Saving to: ‘vgg16_imagenet.h5.1’

vgg16_imagenet.h5.1 100%[===================>]  56.18M  64.7MB/s    in 0.9s    

2024-05-10 02:34:25 (64.7 MB/s) - ‘vgg16_imagenet.h5.1’ saved [58909648/58909648]



## **Run your model**

First, navigate to `code` folder.

In [ ]:
%cd code

Use the provided `main.py` to train and test your models. The provided `argparse` will automatically log and checkpoint your models in newly created `logs/`, `checkpoints/your_model` and `checkpoints/vgg_model` directories. Run the following command to see all available command line arguments.  

In [2]:
!python main.py -h

usage: main.py [-h] --task {1,3} [--data DATA] [--load-vgg LOAD_VGG]
               [--load-checkpoint LOAD_CHECKPOINT] [--confusion] [--evaluate]
               [--lime-image LIME_IMAGE]

Let's train some neural nets!

optional arguments:
  -h, --help            show this help message and exit
  --task {1,3}          Which task of the assignment to run - training from
                        scratch (1), or fine tuning VGG-16 (3). (default:
                        None)
  --data DATA           Location where the dataset is stored. (default:
                        ../data/)
  --load-vgg LOAD_VGG   Path to pre-trained VGG-16 file (only applicable to
                        task 3). (default: vgg16_imagenet.h5)
  --load-checkpoint LOAD_CHECKPOINT
                        Path to model checkpoint file (should end with the
                        extension .h5). Checkpoints are automatically saved
                        when you train your model. If you want to continue
                  

### **Running the tasks**

#### **Task 1 and 3**

The following section uses task 1 as an example. You should be able to run task 3 by replacing `task 1` with `task 3`, and `your_model` with `vgg_model` in the following section.

Use the following command to run a task.

In [3]:
!python main.py --task 3

Dataset mean shape: [48, 48, 3]
Dataset mean top left pixel value: [0.4900, 0.4900, 0.4900]
Dataset std shape: [48, 48, 3]
Dataset std top left pixel value: [0.3248, 0.3248, 0.3248]
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Model: "vgg_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     

**Choosing a checkpoint**

You can load from a specific checkpoint if your current model architecture is the **same** as your model at that checkpoint.

View your checkpoints timestamps and pick one from the list below. The last folder in the list contains your latest training results.

In [20]:
%ls checkpoints/vgg_model

051024-023149/ 051024-023516/


Replace [TIMESTAMP] below with the checkpoint timestamp you've chosen to get a list of model weights.

In [21]:
# TODO: replace [TIMESTAMP] with the timestamp you choose from the last step
%ls checkpoints/vgg_model/051024-023516

vgg.weights.e006-acc0.4726.h5  vgg.weights.e013-acc0.4923.h5
vgg.weights.e007-acc0.4886.h5  vgg.weights.e014-acc0.4964.h5
vgg.weights.e011-acc0.4901.h5


Choose a model from the list above and replace [TIMESTAMP]/[MODEL_FILENAME] with your timestamp and checkpoint file. The last file contains your latest training weights. If you want, you can use the following command to continue training the model with specific weights.

In [14]:
# TODO: replace [TIMESTAMP][MODEL_FILENAME] with the a timestamp from above list
!python main.py --task 3 --load-checkpoint checkpoints/vgg_model/051024-023516/vgg.weights.e014-acc0.4964.h5

Dataset mean shape: [48, 48, 3]
Dataset mean top left pixel value: [0.4605, 0.4605, 0.4605]
Dataset std shape: [48, 48, 3]
Dataset std top left pixel value: [0.3135, 0.3135, 0.3135]
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Model: "vgg_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     

**Testing your model**

Run the following command to evaluate your model. Follow the "choosing your model" section to replace [TIMESTAMP][MODEL_FILENAME].

In [22]:
# TODO: replace [TIMESTAMP][MODEL_FILENAME] with the a checkpoint
!python main.py --task 3 --load-checkpoint checkpoints/vgg_model/051024-023516/vgg.weights.e014-acc0.4964.h5 --evaluate

Dataset mean shape: [48, 48, 3]
Dataset mean top left pixel value: [0.4732, 0.4732, 0.4732]
Dataset std shape: [48, 48, 3]
Dataset std top left pixel value: [0.3286, 0.3286, 0.3286]
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Model: "vgg_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     

#### **Task 2**


**View Tensorboard Logs**

- **Scalars** show how the loss and metrics change with every epoch. You can use them to also track training speed, learning rate, and other scalar values. Scalars can be found in the **Time Series** or **Scalars** dashboards.
- **Graphs** help you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly. Graphs can be found in the **Graphs** dashboard.
- **Histograms** and **Distributions** show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way. Histograms can be found in the **Time Series** or **Histograms** dashboards. Distributions can be found in the **Distributions** dashboard.


In [23]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 31718), started 1:14:52 ago. (Use '!kill 31718' to kill it.)

**Lime Interpreter**

Choose an image your model falsely classified and add it to the end of the command below (replace misclassified/your_model/Bedroom/Store_predicted.png with the path to your image)


**Where to find misclassified images?**
- Under your code folder, find folder **misclassfied**. In the folder you can look for the misclassifed images in their groundtruth category folder, with their names explaining what they are misclassified as. Pick an example you like.

In [26]:
# TODO: replace the image path with the falsely classified image name
# TODO: replace [TIMESTAMP][MODEL_FILENAME] with the a checkpoint
# !python main.py --task 3 --load-checkpoint checkpoints/vgg_model/051024-023516/vgg.weights.e014-acc0.4964.h5 --evaluate --lime-image misclassified/vgg_model/data/surprised_predicted.png
!python main.py --task 3 --load-checkpoint checkpoints/vgg_model/051024-023516/vgg.weights.e014-acc0.4964.h5 --evaluate --lime-image /misclassified/vgg_model/051024-023516/angry/fearful_predicted.png

Dataset mean shape: [48, 48, 3]
Dataset mean top left pixel value: [0.4619, 0.4619, 0.4619]
Dataset std shape: [48, 48, 3]
Dataset std top left pixel value: [0.3294, 0.3294, 0.3294]
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Model: "vgg_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     

**[Optional] Display Lime Interpreter Images**

In [ ]:
# In order to see the images, replace <TIMESTAMP> with your own timestamp
for i in range(4):
  plot_img = imread('lime_explainer_images/051024-023516/' + str(i) + '.png')
  plt.imshow(plot_img)
  plt.show()